In [4]:
# -*- coding:utf-8 -*-
import pymongo
import etl_utils

# 插入一条数据到 result_collection
def insert_one2collection(rslt_coll, doc):
    rslt_coll.insert_one(doc)

# 删除指定collection, 当前未使用
def drop_collection(rslt_coll):
    coll_name = rslt_coll._Collection__name
    db = etl_utils.get_mongo_db()
    ret = db.drop_collection(coll_name)
    print('drop % status:' % coll_name, ret)
    
# 导出数据到result_jd_proidlist
def export2result_jd_proidlist(coll_src, coll_targ):
    coll_src_name = coll_src._Collection__name
    minid = etl_utils.get_start_id(coll_src_name)
    maxid = minid    
    cursor = coll_src.find({'_id': {'$gt': minid}}).sort("_id", pymongo.ASCENDING)
    for cur in cursor:
        doc = { 'phoneid': '',
                'proid': cur['proid'],
                'phonename': '',
                'name': cur['name'],
                'modeltype': '',
                'comment_total':cur['comment_total'],
                'color': cur['color']} 
        maxid = cur['_id']
        insert_one2collection(coll_targ, doc)
        
    etl_utils.set_coll_export2result_log(coll_src_name, minid, maxid)


# 导出数据到result_jd_comment    
def export2result_jd_comment(coll_src, coll_targ):
    coll_src_name = coll_src._Collection__name
    minid  = etl_utils.get_start_id(coll_src_name)
    maxid = minid
    cursor = coll_src.find({'_id': {'$gt': minid}}).sort("_id", pymongo.ASCENDING)
    for cur in cursor:
        words = cur['comment_word']
        proid = cur['proid']
        if words.strip() == '' or proid.strip() == '': continue
        doc = { 'phoneid': '',
                'proid': proid,
                'userid': cur['comment_user'],
                'userlevel': cur['comment_userLeve'],
                'province': '',
                'terminal': '',
                'comment_words': words,
                'comment_score': str(cur['comment_score']),
                'comment_date':cur['comment_date']}   
        maxid = cur['_id']
        insert_one2collection(coll_targ, doc)
        
    etl_utils.set_coll_export2result_log(coll_src_name, minid, maxid)
    
    
# 导出数据到result_jd collection
def export2result_jd(is_appenddate=False):
    print('start etl jd...')
    coll_crawl_proids = etl_utils.get_coll_crawl_proidlist('crawl_jd_proidlist')
    coll_crawl_comment = etl_utils.get_coll_crawl_comment('crawl_jd_comment')    
    coll_rstl_proids = etl_utils.get_coll_result_proidlist('result_jd_proidlist', is_appenddate)
    coll_rstl_comment = etl_utils.get_coll_result_comment('result_jd_comment', is_appenddate)
    
    export2result_jd_proidlist(coll_crawl_proids, coll_rstl_proids)
    export2result_jd_comment(coll_crawl_comment, coll_rstl_comment) 
    print("etl jd ended.")
    
# 清空jd的，etl导出result与log
def clean_jd_exported_result():
    etl_utils.clean_coll_exported_result('crawl_jd_proidlist')
    etl_utils.clean_coll_exported_result('crawl_jd_comment')
    
if __name__ == '__main__':
    is_appenddate = etl_utils.is_export_new_result_coll()
    clean_jd_exported_result()
    export2result_jd(is_appenddate)
    

please input(y/n), is export to new result collection:n
drop result_jd_proidlisttatus: {'ns': 'phone.result_jd_proidlist', 'nIndexesWas': 1, 'ok': 1.0}
drop result_jd_proidlist_20170613tatus: {'ns': 'phone.result_jd_proidlist_20170613', 'nIndexesWas': 1, 'ok': 1.0}
drop result_jd_commenttatus: {'ns': 'phone.result_jd_comment', 'nIndexesWas': 1, 'ok': 1.0}
drop result_jd_comment_20170613tatus: {'ns': 'phone.result_jd_comment_20170613', 'nIndexesWas': 1, 'ok': 1.0}
start etl jd...
result coll: result_jd_proidlist
result coll: result_jd_comment
crawl_jd_proidlist  maxid: 000000000000000000000000
crawl_jd_comment  maxid: 000000000000000000000000
etl jd ended.
